In [29]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd
import openai

## Peek into the data

In [15]:
df_items = pd.read_json("../../data/meta_Clothing_Stores_and_Jewelry_2022_2023_with_category_ratings_100_sample_500.jsonl", lines=True)
df_items.shape

(500, 16)

In [24]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,AMAZON FASHION,Geifa Cardigan for Women Lightweight Ruffle Dr...,4.3,203,"[✦Material: 98% Polyester, 2% Spandex. The Ski...",[],10.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Geifa Cardigan for Women. ', 'url'...",Geifa,"[Clothing, Shoes & Jewelry, Women, Clothing, S...","{'Department': 'womens', 'Date First Available...",B0B6FM17CT,NaN,NaN,NaN
1,AMAZON FASHION,Bruno Marc Men's Dress Tuxedo Shoe Slip-on Cla...,4.7,467,"[100% Velvet, Rubber sole, Greater Stability: ...",[],41.39,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Bruno Marc Men's Dress Tuxedo Shoe...,Bruno Marc,"[Clothing, Shoes & Jewelry, Men, Shoes, Loafer...","{'Date First Available': 'May 7, 2022'}",B09ZQL5X11,NaN,NaN,NaN
2,AMAZON FASHION,"LiotXecd Womens Thongs,Lace Thong Plus Size Pa...",4.5,1537,"[95% Cotton /5% Spandex, 进口, 90percent Cotton,...",[This women's thong is all made of lace. Cotto...,16.98,[{'thumb': 'https://m.media-amazon.com/images/...,[],LiotXecd,"[Clothing, Shoes & Jewelry, Women, Clothing, L...","{'Department': 'Womens', 'Date First Available...",B0B2449BTT,NaN,NaN,NaN
3,AMAZON FASHION,Tanming Women's Vintage High Waist Denim Skirt...,3.9,559,"[Fabric: 90% Cotton, 10% Polyester, Imported, ...",[],36.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],Tanming,"[Clothing, Shoes & Jewelry, Women, Clothing, S...","{'Department': 'womens', 'Date First Available...",B0B2JLHFQ9,NaN,NaN,NaN
4,AMAZON FASHION,Branduce Mens Solid Color Double Fold Pre-tied...,4.6,435,"[silk, Imported, Tie closure, Dry Clean Only, ...",[],8.94,[{'thumb': 'https://m.media-amazon.com/images/...,[],Branduce,"[Clothing, Shoes & Jewelry, Men, Accessories, ...","{'Date First Available': 'May 23, 2022'}",B0B21S83Y2,NaN,NaN,NaN


In [ ]:
df_items.info()

In [67]:
df_items["categories"]

0      [Clothing, Shoes & Jewelry, Women, Clothing, S...
1      [Clothing, Shoes & Jewelry, Men, Shoes, Loafer...
2      [Clothing, Shoes & Jewelry, Women, Clothing, L...
3      [Clothing, Shoes & Jewelry, Women, Clothing, S...
4      [Clothing, Shoes & Jewelry, Men, Accessories, ...
                             ...                        
495    [Clothing, Shoes & Jewelry, Men, Watches, Wris...
496    [Clothing, Shoes & Jewelry, Novelty & More, Cl...
497    [Clothing, Shoes & Jewelry, Women, Accessories...
498    [Clothing, Shoes & Jewelry, Women, Clothing, T...
499    [Clothing, Shoes & Jewelry, Women, Shoes, Fash...
Name: categories, Length: 500, dtype: object

In [ ]:
df_items['features'][0]

["✦Material: 98% Polyester, 2% Spandex. The Skin-Friendly fabric is soft and lightweight with stretch, won't be itchy, comfortable to wear all year round.",
 'Imported',
 'Pull On closure',
 'Machine Wash',
 "✦Feminine Design: Long Sleeve open front cardigan for women, ruffled drapes cardigan sweaters with irregular hem, light fall jackets. It's soft and not too heavy, an essential for daily all seasons.",
 '✦Match: The cardigan sweater is a perfect addition to any outfit, easy to pair with tank tops/shirts /jeans/dresses/pants/leggings/boots to elongate your silhouette and flatter your figure.',
 '✦Occasion: The long cardigans can be worn for all seasons and versatile for any occasions. Suitable for casual daily or as a Lightweight Blazers for work, business, office, wedding, beach, travel and so on. It is an ideal choice as gift to your friends, families or yourself.',
 '✦Clothing Care: Machine or Hand Wash with Cold Water, Better to wash in a laundry bag/Hang or Line Dry/No Bleach/N

In [29]:
df_items['description']

0                                                     []
1                                                     []
2      [This women's thong is all made of lace. Cotto...
3                                                     []
4                                                     []
                             ...                        
495                                                   []
496    [It's dangerous to go alone, take a Legend of ...
497                                                   []
498    [Quality clothes for all the good things ahead...
499    [Beach Bingo - Kitten Bitten Lovergirl scrunch...
Name: description, Length: 500, dtype: object

In [34]:
df_items["images"][0]

[{'thumb': 'https://m.media-amazon.com/images/I/41maGFZPjTL._AC_SR38,50_.jpg',
  'large': 'https://m.media-amazon.com/images/I/41maGFZPjTL._AC_.jpg',
  'variant': 'MAIN',
  'hi_res': 'https://m.media-amazon.com/images/I/81EImKgL-xL._AC_UL1500_.jpg'},
 {'thumb': 'https://m.media-amazon.com/images/I/41IZ64I12zL._AC_SR38,50_.jpg',
  'large': 'https://m.media-amazon.com/images/I/41IZ64I12zL._AC_.jpg',
  'variant': 'PT01',
  'hi_res': 'https://m.media-amazon.com/images/I/812W0z5SvvL._AC_UL1500_.jpg'},
 {'thumb': 'https://m.media-amazon.com/images/I/41IF5iAvlRL._AC_SR38,50_.jpg',
  'large': 'https://m.media-amazon.com/images/I/41IF5iAvlRL._AC_.jpg',
  'variant': 'PT02',
  'hi_res': 'https://m.media-amazon.com/images/I/71jZA3MBuxL._AC_UL1500_.jpg'},
 {'thumb': 'https://m.media-amazon.com/images/I/31wwvO2tQ1L._AC_SR38,50_.jpg',
  'large': 'https://m.media-amazon.com/images/I/31wwvO2tQ1L._AC_.jpg',
  'variant': 'PT03',
  'hi_res': 'https://m.media-amazon.com/images/I/71hw2lfKCML._AC_UL1500_.jpg

In [43]:
df_items["videos"][0]

[{'title': 'Geifa Cardigan for Women. ',
  'url': 'https://www.amazon.com/vdp/04dfd0f9a02c4639a38814db1f4bc7e6?ref=dp_vse_rvc_0',
  'user_id': '/shop/shonatay7'}]

In [ ]:
df_items["details"][0]

{'Department': 'Womens', 'Date First Available': 'May 24, 2022'}

## Transformation
1.Convert features,description and details into strings and concatenate with title, categories

2.Extract large image from images column

3.Extract video url from videos column


In [16]:
def preprocess_description(row):
    return f"{row["title"]} {" ".join(row["features"])} {" ".join(row["description"])} {" ".join(row["details"])} {" ".join(row["categories"])}"

In [17]:
def extract_large_image(row):
    return f"{row["images"][0].get("large","")}"

In [18]:
def extract_video_url(row):
    # Check if videos list exists and has at least one element
    if row['videos'] and len(row['videos']) > 0:
        return row['videos'][0].get('url', '')
    return ''

In [19]:
def preprocess_details(row):
    if not isinstance(row, dict):
        return "" 
    return ". ".join([f"{key}: {value}" for key, value in row.items()])


In [20]:
df_items['details'] = df_items['details'].apply(preprocess_details)
df_items["description"] = df_items.apply(preprocess_description,axis=1)
df_items["image"] = df_items.apply(extract_large_image, axis=1)
df_items["video"] = df_items.apply(extract_video_url, axis=1)


In [8]:
df_items["description"][0]

"Geifa Cardigan for Women Lightweight Ruffle Drape Open Front Cardigans Sweaters Long Sleeve ✦Material: 98% Polyester, 2% Spandex. The Skin-Friendly fabric is soft and lightweight with stretch, won't be itchy, comfortable to wear all year round. Imported Pull On closure Machine Wash ✦Feminine Design: Long Sleeve open front cardigan for women, ruffled drapes cardigan sweaters with irregular hem, light fall jackets. It's soft and not too heavy, an essential for daily all seasons. ✦Match: The cardigan sweater is a perfect addition to any outfit, easy to pair with tank tops/shirts /jeans/dresses/pants/leggings/boots to elongate your silhouette and flatter your figure. ✦Occasion: The long cardigans can be worn for all seasons and versatile for any occasions. Suitable for casual daily or as a Lightweight Blazers for work, business, office, wedding, beach, travel and so on. It is an ideal choice as gift to your friends, families or yourself. ✦Clothing Care: Machine or Hand Wash with Cold Wate

In [9]:
df_items["image"][0]

'https://m.media-amazon.com/images/I/41maGFZPjTL._AC_.jpg'

In [10]:
df_items['video'][0]

'https://www.amazon.com/vdp/04dfd0f9a02c4639a38814db1f4bc7e6?ref=dp_vse_rvc_0'

In [11]:
df_items["details"][0]

'Department: womens. Date First Available: September 12, 2022'

## Sample the data

In [22]:
df_sample = df_items.sample(50, random_state=45)

In [23]:
df_sample.shape

(50, 18)

## Embeddings

In [25]:
data_to_embed = df_sample[["parent_asin","description","price", "average_rating", "rating_number", "image", "video", "store"]].to_dict(orient="records")

In [28]:
data_to_embed

[{'parent_asin': 'B0BF9GZFBQ',
  'description': "Moyabo Women's Knit Ribbed Maternity Dress Long Sleeve Bodycon Dress for Daily Wearing or Baby Shower ✿ 34%Cotton+62%Polyester+4%Spandex. Imported Pull On closure Hand Wash Only ✿ The maternity dress is super soft, stretchy, comfortable, skin-friendly, breathable, high-quality. The womens dresses provide plenty of room for your growing pregnant belly and available in a vatiety of colors.You can wear them throughout your pregnancy and beyond. ✿ Classic Elegant Simple Style-This bodycon pregnancy dress is features with turtle neck, long sleeve, knee length, midi bodycon maternity dress. It's super versatile, easy to style, great with coats, jacket, trench coat, leggings, boots, very charming and feminine. ✿ Show Off Your Bump-The knit ribbed maternity dress flatters the body so beautiful it accentuates your curves show your great slim fit and good temperament through all stages of your pregnancy. ✿ Maternity Dress can be used pre and post 

In [27]:
def get_embeddings(text,model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

## Initialize Qdrant and create qdrant collection

In [36]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [ ]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

## Create pointstructs to upload/upsert data to qdrant

In [41]:
pointstruct = []
for i,data in enumerate(data_to_embed):
    pointstruct.append(
        PointStruct(
            id=i,
            vector=get_embeddings(data["description"]),
            payload=data
        )
    )

In [42]:
pointstruct

[PointStruct(id=0, vector=[0.06161314621567726, 0.049624886363744736, -0.05945199728012085, 0.027095921337604523, -0.01999063231050968, -0.015484839677810669, 0.006401895545423031, -0.0003329011087771505, -0.015994545072317123, -0.02538331225514412, 0.0074875676073133945, -0.026300780475139618, -0.04191814363002777, -0.027585238218307495, -0.0034914796706289053, 0.051663704216480255, -0.01941976323723793, -0.021428000181913376, -0.03651526942849159, 0.06181703135371208, 0.014587759040296078, 0.02848231978714466, 0.045791901648044586, -0.04823848605155945, 0.047830723226070404, 0.021529942750930786, -0.00317546259611845, -0.02848231978714466, 0.007105288561433554, -0.023201774805784225, -0.025220206007361412, -0.008695567958056927, 0.0004937767516821623, -0.04897246137261391, 0.0011041485704481602, -0.008756732568144798, -0.032437629997730255, 0.004809067118912935, 0.050277307629585266, -0.006412089802324772, -0.005224477034062147, 0.0038661127910017967, 0.045424915850162506, -0.0301541

## Write embedded data to qdrant

In [43]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstruct
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

## Define retrieval function

In [48]:
def retrieve_data(query,k):
    result = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=get_embeddings(query),
        limit=7
    )
    return result

## Test the retrieval

In [50]:
retrieve_data("what kind of shoes can I get?", k=10).points

[ScoredPoint(id=48, version=0, score=0.40231344, payload={'parent_asin': 'B0B17XG5V7', 'description': 'Ermonn Women\'s Slingback Chunky Heels Closed Toe Low Heel Pumps Woven Knit Strap Backless Mules Casual Dress Shoes Rubber sole ❤ [ HEEL HEIGHT ]: Self-covered low heel, 1.7" heel/4.5cm (approx) ❤ [ MATERIAL ]: Faux Suede/ Woven Knit Textile Material Upper; Rubber Sole. The upper is soft and not easily deformed, the inside is breathable and comfortable, suitable for spring summer and autumn ❤ [ FEATURES ]: Almond toe; chunky block heel; woven knit upper; pulls on; elastic back strap; backless shoes. Chic and comfy low heel mules, more breathable, lightweight and stable than regular heels and you can walk easily for long periods. Elastic back straps, better fixation, less likely to fall off, and less likely to rub your feet ❤ [ MATCH ]: The best choice for this autumn, stylish casual style, simple, classic and unique, they can go with everything, look great with jeans, slacks, jackets 